In [1]:
import numpy as np
import torch
import torch.nn.functional as F

Доп задание

In [52]:
# Изменим функцию из 1 лабораторной работы
def conv2D_my_help(input, weight, stride, padding,output_padding):
    batch_size, in_channels, img_height, img_width = input.shape
    out_channels, in_channels, kernel_height, kernel_width = weight.shape
    h_out = (img_height - 1) * stride - 2 * padding + kernel_height + output_padding
    w_out = (img_width - 1) * stride - 2 * padding + kernel_width + output_padding

    result = np.zeros((batch_size, out_channels,h_out, w_out))

    return (batch_size, out_channels, h_out, w_out , 
            in_channels, kernel_height, 
            kernel_width, img_height, img_width, result)

In [54]:
# Реализуем транспонированноую свёртку
def conv2D_T_my(input, weight, bias=None, stride=1, padding=0, output_padding=0, groups=1, dilation=1):
    
    (batch_size, out_channels, H_out, 
    W_out, in_channels,
    kernel_height, kernel_width,
    in_height, in_width, result) = conv2D_my_help(input, weight,
                                                       stride, padding, output_padding=output_padding)
    
    if padding > 0:
        input = np.pad(input, padding, mode='constant')

    result = np.zeros((batch_size, out_channels, H_out, W_out))


    for batch in range(batch_size):
        for channel_o in range(out_channels):
            for h_o in range(H_out):
                for w_o in range(W_out):
                    for in_c in range(in_channels):
                        for kern_h in range(kernel_height):
                            for kern_w in range(kernel_width):
                                ii = h_o + padding - kern_h * dilation
                                jj = w_o + padding - kern_w * dilation
                                if ii >= 0 and jj >= 0 and ii < in_height * stride and jj < in_width * stride and (ii % stride == 0) and (jj % stride == 0):
                                    ii //= stride
                                    jj //= stride
                                    result[batch, channel_o, h_o, w_o] += np.multiply(input[batch, in_c, ii, jj], weight[channel_o, in_c, kern_h, kern_w])
    return result


Основное задание

In [43]:
def customConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0\
                          , output_padding=0, bias=True, dilation=1, padding_mode='zeros'):
    #Обёртка
    def wrapper(mat):
        
        #Проверки параметров
        if output_padding < 0:
            raise Exception(f"Padiing should be 0 or positive")
        if stride < 0:
            raise Exception(f"Stride should be 0 or positive")
        if (padding_mode != 'zeros'):
            raise Exception(f"Ivalid padding_mode")
            
        #Смещение
        if bias:
            bias_value = torch.rand(out_channels)
        else:
            bias_value = torch.zeros(out_channels)
            
        
        #Фильтр с учётом размера ядра
        if type(kernel_size) == tuple:
            flter = torch.rand(in_channels, out_channels, kernel_size[0], kernel_size[1])
        elif type(kernel_size) == int:
            flter = torch.rand(in_channels, out_channels, kernel_size, kernel_size)
        else:
            raise Exception(f"Ivalid kernel_size type")
            
            
        #"Обход" фильтром
        res = []
        for ochnl in range(out_channels):
            feature_map = torch.zeros((mat.shape[1] - 1) * stride + dilation * (flter.shape[2] - 1) + 1\
                                            , (mat.shape[2] - 1) * stride + dilation * (flter.shape[3] - 1) + 1)
            
            for ichnl in range(in_channels):
                for i in range(0, mat.shape[1]):
                    for j in range(0, mat.shape[2]):
                        cur = mat[ichnl][i][j]
                        val = cur * flter[ichnl][ochnl]
                        zeros = torch.zeros((flter.shape[2] - 1) * dilation + 1, (flter.shape[3] - 1) * dilation + 1)
                        for k in range(0, zeros.shape[0], dilation):
                            for f in range(0, zeros.shape[1], dilation):
                                zeros[k][f] = val[k // dilation][f // dilation]
                        total = np.add((zeros), feature_map[i * stride:i * stride + dilation * (flter.shape[2] - 1) + 1\
                                                            , j * stride:j * stride + dilation * (flter.shape[3] - 1) + 1])
                        
                        feature_map[i * stride:i * stride + dilation * (flter.shape[2] - 1) + 1\
                                    , j * stride:j * stride + dilation * (flter.shape[3] - 1) + 1] = total

            res.append(np.add(feature_map, np.full((feature_map.shape), bias_value[ochnl])))
        
        for l in range(len(res)):
            if output_padding > 0:
                pad = torch.nn.ConstantPad1d((0, output_padding, 0, output_padding), 0)
                res[l] = pad(res[l])
            res[l] = res[l][padding:res[l].shape[0] - padding, padding:res[l].shape[1] - padding]
            
        return np.array(res), np.array(flter), np.array(bias_value)
    return wrapper

Тесты основного

In [44]:
test_data_1 = torch.rand(3, 28, 28)



def test1():
    customConv = customConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0\
                                      , output_padding=0, bias=True, dilation=3, padding_mode='zeros')
    
    result, flter, bias_value = customConv(test_data_1)
    torchConv = torch.nn.ConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0\
                                      , output_padding=0, bias=True, dilation=3, padding_mode='zeros')
    
    torchConv.weight.data = torch.tensor(flter)
    torchConv.bias.data = torch.tensor(bias_value)
    customResult = str(np.round(result,2))
    torchResult = str(np.round(np.array(torchConv(test_data_1).data),2))
    assert customResult == torchResult

In [ ]:
test1()

Тесты доп задания

In [34]:
image = torch.randn(1, 1, 5, 5) 
weight = torch.randn(1, 1, 3, 3)
    
myConv2D_T = torch.from_numpy(conv2D_T_my(image.numpy(), weight.numpy()))


torchConv2D_T = F.conv_transpose2d(image, weight)

myConv2D_T = myConv2D_T.to(torchConv2D_T.dtype)

print(myConv2D_T)
print(torchConv2D_T)

tensor([[[[-0.0701,  0.2319, -1.1459,  1.6100, -0.5627,  4.1986, -2.3870],
          [-0.1406,  0.5679,  0.0813,  1.4064, -0.0839,  8.1608, -2.8441],
          [-0.3351, -0.0324,  0.1492,  5.7634, -1.4600,  2.5127, -1.8647],
          [ 0.2987,  1.5988,  1.2388,  7.9291,  3.1784, -1.8894,  1.4940],
          [-0.2612,  0.3133, -0.8188, -4.7213, -0.1466, -5.3322,  2.0261],
          [ 0.2582,  2.7574,  1.4441, -1.7009,  0.6177, -0.4861,  0.7607],
          [-1.5094, -4.4170, -4.4054, -3.3296, -2.0211,  0.5816,  0.1466]]]])
tensor([[[[-0.0701,  0.2319, -1.1459,  1.6100, -0.5627,  4.1986, -2.3870],
          [-0.1406,  0.5679,  0.0813,  1.4064, -0.0839,  8.1608, -2.8441],
          [-0.3351, -0.0324,  0.1492,  5.7634, -1.4600,  2.5127, -1.8647],
          [ 0.2987,  1.5988,  1.2388,  7.9291,  3.1784, -1.8894,  1.4940],
          [-0.2612,  0.3133, -0.8188, -4.7213, -0.1466, -5.3322,  2.0261],
          [ 0.2582,  2.7574,  1.4441, -1.7009,  0.6177, -0.4861,  0.7607],
          [-1.5094, -4

In [58]:
def test_1():
    image = torch.randn(1, 1, 5, 5) 
    weight = torch.randn(1, 1, 3, 3)
    
    myConv2D_T = torch.from_numpy(conv2D_T_my(image.numpy(), weight.numpy()))


    torchConv2D_T = F.conv_transpose2d(image, weight)

    myConv2D_T = myConv2D_T.to(torchConv2D_T.dtype)

    assert torch.allclose(myConv2D_T, torchConv2D_T)

def test_2():
    image = torch.randn(1, 1, 7, 7) 
    weight = torch.randn(1, 1, 2, 2) 
    
    myConv2D_T = torch.from_numpy(conv2D_T_my(image.numpy(), weight.numpy()))


    torchConv2D_T = F.conv_transpose2d(image, weight)

    myConv2D_T = myConv2D_T.to(torchConv2D_T.dtype)

    assert torch.allclose(myConv2D_T, torchConv2D_T)

def test_3():
    image = torch.randn(1, 1, 4, 4) 
    weight = torch.randn(1, 1, 4, 4) 
    
    myConv2D_T = torch.from_numpy(conv2D_T_my(image.numpy(), weight.numpy()))

    torchConv2D_T = F.conv_transpose2d(image, weight)

    myConv2D_T = myConv2D_T.to(torchConv2D_T.dtype)

    assert torch.allclose(myConv2D_T, torchConv2D_T)

def test_4():
    image = torch.randn(1, 1, 6, 6) 
    weight = torch.randn(1, 1, 2, 2) 
    
    myConv2D_T = torch.from_numpy(conv2D_T_my(image.numpy(), weight.numpy()))

    torchConv2D_T =F.conv_transpose2d(image, weight)

    myConv2D_T = myConv2D_T.to(torchConv2D_T.dtype)

    assert torch.allclose(myConv2D_T, torchConv2D_T)

# Успешный тест

In [30]:
try:
    test_1()
    test_2()
    test_3()
    test_4()
    print('ok')
except BaseException as e:
    print('error')
    print(e)

ok


# Неуспешный тест

In [28]:
try:
    test_1()
    test_2()
    test_3()
    test_4()
    print('ok')
except BaseException as e:
    print('error')
    print(e)

error
The size of tensor a (7) must match the size of tensor b (12) at non-singleton dimension 3
